In [25]:
!pwd
!ls ../../generated_epgs/

/Users/amarmesic/Documents/tudelft/thesis/dna/TGAN/Generator
discrimination_test
fixed_ratios_base_template_500_5000_improved
fixed_ratios_base_template_500_5000_improved_modelled
fixed_ratios_base_template_500_5000_no_preprocess
fixed_ratios_base_template_500_5000_no_preprocess.zip
fixed_ratios_base_template_500_5000_scaled_normal_z
fixed_ratios_base_template_500_5000_scaled_uniform_z-BEST


In [20]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os

In [21]:
def plot_epg_plotly(epg: np.ndarray, no_dyes=6, title="EPG", save_path="epg_plot",
                    include_lane_standard=True, save_static_pdf=True):
    """
    Plot electropherogram using Plotly with optional saving as HTML and static image.

    Parameters:
        epg (np.ndarray): EPG array of shape (1, no_dyes, scan_points, 1)
        no_dyes (int): Number of dye channels (default: 6)
        title (str): Title of the plot
        save_path (str): Path (without extension) for saving output
        include_lane_standard (bool): Whether to include internal lane standard channel
        save_static_pdf (bool): Whether to save a static PDF version (requires kaleido)
    """
    lane_colors = ["magenta", "blue", "cyan", "green", "yellow", "orange"]
    lanes = no_dyes if include_lane_standard else no_dyes - 1

    fig = make_subplots(rows=lanes, cols=1, shared_xaxes=True,
                        vertical_spacing=0.02, subplot_titles=[f"Dye Channel {i+1}" for i in range(lanes)])
    
    for i in range(lanes):
        signal = epg[i, :, 0]
        fig.add_trace(
            go.Scatter(
                x=np.arange(len(signal)),
                y=signal,
                mode='lines',
                line=dict(color=lane_colors[i], width=1),
                name=f"Dye {i+1}"
            ),
            row=i + 1,
            col=1
        )
        fig.update_yaxes(title_text="Intensity (RFU)", row=i + 1, col=1)

    fig.update_xaxes(title_text="Scan Points", row=lanes, col=1)
    fig.update_layout(height=250 * lanes, width=1000,
                      title_text=title,
                      hovermode="x unified",
                      showlegend=False)
    
    # Show the interactive figure
    fig.show()

    # Save as interactive HTML
    fig.write_html(f"{save_path}.html")

In [47]:
# Directory containing .npy files
npy_dir = '../../generated_epgs/fixed_ratios_base_template_500_5000_scaled_uniform_z_1-2/epgs'
limit = 2  # Number of arrays to load (like head)

# List all .npy files in the directory
npy_files = [f for f in os.listdir(npy_dir) if f.endswith('.npy')]
# sort the files by name to ensure consistent order
npy_files.sort()
npy_files = npy_files[:limit]

# Load arrays and keep filenames
arrays = [(np.load(os.path.join(npy_dir, fname)), fname) for fname in npy_files]

print(f"Loaded {len(arrays)} arrays."), [arr[1] for arr in arrays]

Loaded 2 arrays.


(None,
 ['epg_configID10_sample_0001_N2_167_333_fe_rep1_deg2.npy',
  'epg_configID10_sample_0001_N2_167_333_fe_rep2_deg2.npy'])

In [48]:
for arr, fname in arrays:
    plot_epg_plotly(arr, title=f"EPG {fname}")